In [33]:
import torch
import numpy as np

In [2]:
x = torch.rand(5, 3)

In [4]:
y = torch.rand(3, 2)

In [6]:
x @ y

tensor([[0.5996, 0.9200],
        [0.3115, 1.0785],
        [0.5918, 0.9897],
        [0.5772, 1.5284],
        [0.5121, 1.1045]])

In [16]:
t = torch.tensor([1, 2])

In [20]:
t.numel()

2

In [21]:
t.shape

torch.Size([2])

In [22]:
t.ndim

1

In [23]:
x.T

tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])

In [14]:
x.device

device(type='cpu')

In [24]:
x.dtype

torch.float32

In [32]:
x.numpy()

array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)

In [36]:
torch.from_numpy(np.array([1, 2, 3], dtype=float))

tensor([1., 2., 3.], dtype=torch.float64)